# Companies House

<p>
Mal Minhas, v0.1<br>
27.10.24
</p>
<p>
<h4>Versions</h4>
<ul>
<li><b>v0.1</b>: 27.10.24. First version focussed only on REST API</li>
</ul>
</p>

### 1. Introduction

This playbook provides a brief outline of how to access the Companies House (CH) REST API per the developer documentation available [here](https://developer.company-information.service.gov.uk/overview).  In order to follow along and use this playbook you will first need to [create an application](https://developer.company-information.service.gov.uk/how-to-create-an-application/) and associated API key that you will need to access the CH API.  Note that you can set up applications and keys to communicate with either a Sandbox endpoint which is read/write or the public API which is just read.  These endpoints are located at the following endpoints:
* **Sandbox API endpoint**: test-data-sandbox.company-information.service.gov.uk
* **Public API endpoint**: api.company-information.service.gov.uk

This notebook will focus only on the public read endpoint.

### 1. Authorisation

The CH REST API uses [HTTP Basic Auth](https://en.wikipedia.org/wiki/Basic_access_authentication) for GET requests in a CLI tool.  There is an OAuth2 flow as well which appears to be required for certain types of sensitive access. For instance obtaining [user profile](https://developer-specs.company-information.service.gov.uk/companies-house-identity-service/reference/user-details/user-profile) information from the endpoint at https://identity.company-information.service.gov.uk/user/profile requires going through OAuth2 flow to obtain a `client_id` and `client_secret` per the documentation [here](https://developer-specs.company-information.service.gov.uk/companies-house-identity-service/guides/ServerWeb).  This playbook is going to focus on the simple REST API access use case only rather than the web flow one which is more involved as it requires setting up a web server to handle the redirect.

### 2. Company Profile

Let's start with the most basic information which relates to company profile.  The following function `getCompanyInfo` will retrieve that information:

In [1]:
import os
import requests

API_KEY = os.environ.get('COMPANIES_HOUSE_API_KEY')

def procUrl(url, query_params={}, verbose=False):
    r = requests.get(url, auth=(API_KEY, ''), params=query_params)
    if verbose:
        print("-------------------------")
        print(f"URL: {r.request.url}")
        print(f"Body: {r.request.body}")
        hdrs = ""
        for k,v in r.request.headers.items():
            hdrs += f"  {k}:{v}\n"
        print(f"Headers:\n{hdrs[:-1]}")
        print("-------------------------")
    if r.status_code != 200:
        raise Exception(f"Error {r.status_code}: '{r.text}'")
    return r.json()  # if response is JSON format

def getCompanyProfile(id):
    directive = "company"
    url = f"https://api.company-information.service.gov.uk/{directive}/{id}"
    return procUrl(url)

We can use it to find out information about a company called [Oppian](https://find-and-update.company-information.service.gov.uk/company/06782942) which has a company number 06782942 which we obtained from using the CH web search interface.

In [2]:
def procAddress(data):
    ''' ChatGPT Python GPT assisted :-) '''
    # Extract the registered office address from the data
    address = data.get('registered_office_address', {})
    # Retrieve each part of the address, or an empty string if it doesn't exist
    address_line_1 = address.get('address_line_1', '')
    address_line_2 = address.get('address_line_2', '')
    locality = address.get('locality', '')
    region = address.get('region', '')
    postal_code = address.get('postal_code', '')
    country = address.get('country', '')
    # Concatenate all parts with commas, filtering out empty strings
    full_address = ', '.join(filter(None, [address_line_1, address_line_2, locality, region, postal_code, country]))
    return full_address

companyId = '06782942'
d = getCompanyProfile(companyId)
print(f"{d.get('company_name')} has company id={d.get('company_number')} is {d.get('company_status')} and has registered address:\n{procAddress(d)}")

OPPIAN SYSTEMS LIMITED has company id=06782942 is active and has registered address:
C/O High Royd Business Services Limited B B I C, Innovation Way, Barnsley, South Yorkshire, S75 1JL, United Kingdom


### 3. Company Details

Let's have a look at extracting information about getting a list of [persons with significant control](https://developer-specs.company-information.service.gov.uk/companies-house-public-data-api/reference/persons-with-significant-control/list?v=latest) which is obtained from https://api.company-information.service.gov.uk/company/{company_number}/persons-with-significant-control:

In [3]:
def getPersonsWithSignificantControl(id):
    directive = f"company/{id}/persons-with-significant-control"
    url = f"https://api.company-information.service.gov.uk/{directive}"
    params = {'start_index':0, 'items_per_page': 10, 'register_view':'false'}
    return procUrl(url, params, verbose=False)

d = getPersonsWithSignificantControl(companyId)
print(f"Found {len(d.get('items'))} persons with significant control")

Found 2 persons with significant control


### 4. Search

We can also conduct search programmatically to mirror the search user interface [here](https://find-and-update.company-information.service.gov.uk/):

In [4]:
def searchCompany(category, name, nitems=10, start_index=0):
    directive = f"search/{category}"
    url = f"https://api.company-information.service.gov.uk/{directive}?q={name}&start_index=0"
    return procUrl(url)

In [5]:
d = searchCompany('companies','Oppian')

A cursory look at this data reveals the same information:

In [6]:
company_title = d.get('items')[0].get('title')
company_id = d.get('items')[0].get('company_number')
company_description = d.get('items')[0].get('company_number')
print(f"Company '{company_title}' with company id={company_id} has description {company_description}")

Company 'OPPIAN SYSTEMS LIMITED' with company id=06782942 has description 06782942
